## Load data

In [13]:
import requests

# tiny shakespeare data
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
data = response.text

## Data Exploration

In [14]:
print("length of dataset in characters: ", len(data))

length of dataset in characters:  1115394


In [15]:
print(data[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [16]:
chars = sorted(list(set(data)))
vocab_size = len(chars)
print(''.join(chars))
print('vocab size: ', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size:  65


## Data pre-processing

### Option 1: Tokenize in sub-words

In [17]:
# import tiktoken

# enc = tiktoken.get_encoding('gpt2')
# enc.n_vocab

In [18]:
# print(enc.encode("Hello, World!"))
# print(enc.decode(enc.encode("Hello, World!")))

### Option 2: Tokenize in characters

In [19]:
# create a mapping from characters to integers
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [char_to_int[ch] for ch in s] # encode a string, output is a list of integers
decode = lambda x: ''.join([int_to_char[i] for i in x]) # decode a list of integers, output is a string

print(encode("Hello, World!"))
print(decode(encode("Hello, World!")))

[20, 43, 50, 50, 53, 6, 1, 35, 53, 56, 50, 42, 2]
Hello, World!


### Train-test split

In [20]:
# Put the data into a tensor
import torch

# Move the data to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Encode the data
data_tensor = torch.tensor(encode(data), dtype=torch.long, device=device)

print(data_tensor.shape, data_tensor.dtype)
print(data_tensor[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [21]:
test_size = 0.9 # use 90% of the data for training, 10% for testing
train_data_tensor = data_tensor[:int(test_size*len(data_tensor))]
test_data_tensor = data_tensor[int(test_size*len(data_tensor)):]

In [22]:
block_size = 8
train_data_tensor[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

### Batch data

In [23]:
x = train_data_tensor[:block_size]
y = train_data_tensor[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context.tolist()}, target is {target}")

When input is [18], target is 47
When input is [18, 47], target is 56
When input is [18, 47, 56], target is 57
When input is [18, 47, 56, 57], target is 58
When input is [18, 47, 56, 57, 58], target is 1
When input is [18, 47, 56, 57, 58, 1], target is 15
When input is [18, 47, 56, 57, 58, 1, 15], target is 47
When input is [18, 47, 56, 57, 58, 1, 15, 47], target is 58


In [24]:
torch.manual_seed(1337)
batch_size = 8
block_size = 128

# Generates a small batch (size=batch_size) of data
# The data is stacked in a tensor of shape (batch_size x block_size)

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data_tensor if split == 'train' else test_data_tensor
    ix = torch.randint(len(data) - block_size, (batch_size,)) # starting index for each sequence
    x = torch.stack([data[i:i+block_size] for i in ix]) # input
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # target
    return x, y

xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("------")

for b in range(batch_size):
    print("batch", b)
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()}, target is {target}")

inputs:
torch.Size([8, 128])
tensor([[ 1, 45, 59,  ...,  1, 55, 59],
        [52, 47, 57,  ..., 54, 56, 43],
        [50,  1, 15,  ..., 42, 43, 43],
        ...,
        [16, 33, 23,  ..., 58,  1, 39],
        [ 1, 53, 56,  ...,  0, 32, 46],
        [37, 10,  0,  ..., 54, 53, 57]])
targets:
torch.Size([8, 128])
tensor([[45, 59, 43,  ..., 55, 59, 43],
        [47, 57, 46,  ..., 56, 43, 60],
        [ 1, 15, 53,  ..., 43, 43, 42],
        ...,
        [33, 23, 17,  ...,  1, 39, 57],
        [53, 56,  1,  ..., 32, 46, 43],
        [10,  0, 32,  ..., 53, 57, 57]])
------
batch 0
When input is [1], target is 45
When input is [1, 45], target is 59
When input is [1, 45, 59], target is 43
When input is [1, 45, 59, 43], target is 57
When input is [1, 45, 59, 43, 57], target is 57
When input is [1, 45, 59, 43, 57, 57], target is 1
When input is [1, 45, 59, 43, 57, 57, 1], target is 61
When input is [1, 45, 59, 43, 57, 57, 1, 61], target is 46
When input is [1, 45, 59, 43, 57, 57, 1, 61, 46], tar

## Implement bigram language model

In [25]:
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):

        # idx: (batch_size, block_size) (B, T)
        # targets: (batch_size, block_size) (B, T)
        logits = self.token_embedding(idx) # shape: (batch_size, block_size, vocab_size) (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)

            # How well are we predicting the next token based on the logits?
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, steps):
        # idx: (batch_size, block_size) (B, T)
        for _ in range(steps):
            # get the predictions
            logits, _ = self(idx)
            # get the last time step
            logits = logits[:, -1, :] # shape: (B, C)
            # get the probabilities
            probs = F.softmax(logits, dim=-1) # shape: (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # shape: (B, 1)
            # append to the sequence
            idx = torch.cat((idx, idx_next), dim=1) # shape: (B, T+1)
        return idx

    
model = BigramLanguageModel(vocab_size).to(device)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss) # the expected loss should be close to -log(1/vocab_size) = -ln(1/65) = 4.17

idx = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(idx, steps=100)[0].tolist()))

torch.Size([1024, 65])
tensor(4.7254, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


## Train model

In [26]:
# Create a PyTorch Optimizer
# AdamW is a popular optimizer that is known to work well with transformers
# A learning rate of 1e-3 is a good starting point
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3) 

In [27]:
batch_size = 32
for steps in range(10000):

    # get a batch of data
    xb, yb = get_batch('train')

    # forward pass
    logits, loss = model(xb, yb)

    # backward pass
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % 1000 == 0:
        print(f"Step {steps}, Loss {loss.item()}")

Step 0, Loss 4.729787826538086
Step 1000, Loss 3.547955274581909
Step 2000, Loss 2.918903350830078
Step 3000, Loss 2.6329891681671143
Step 4000, Loss 2.5600321292877197
Step 5000, Loss 2.4919416904449463
Step 6000, Loss 2.4866533279418945
Step 7000, Loss 2.4358408451080322
Step 8000, Loss 2.470460891723633
Step 9000, Loss 2.452385902404785


In [28]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(idx, steps=100)[0].tolist()))


Iyoteng h hasbe pave pirance
RDe hicomyonthar's
PES:
AKEd ith henouratucenonthioneir thondy, y helti


## Self-Attention

In [41]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # Batch, time, channels
x = torch.randn(B,T,C)

# single head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16) public information about token itself
q = query(x) # (B, T, 16) public information that token wants to have
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)


tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # use lower-triangle matrix to remove communication between current token and future tokens
wei = F.softmax(wei, dim=-1) # distribute tokens between (0, 1)

v = value(x) # private information about token
out = wei @ v
#out = wei @ x


out.shape

torch.Size([4, 8, 16])

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [38]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [ ]:
torch.manual_seed(1337)
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5 # (qk^T)/(sqrt(head_size))

In [48]:
k.var()

tensor(0.3164, grad_fn=<VarBackward0>)

In [49]:
q.var()

tensor(0.3386, grad_fn=<VarBackward0>)

In [50]:
wei.var()

tensor(0.0472, grad_fn=<VarBackward0>)